In [2]:
import sys
sys.path.append("..")

from glob import glob
import matplotlib.pyplot as plt
import ipywidgets as ipw
from IPython.display import Audio
import numpy as np
import pickle
import yaml
import pandas as pd

from lib.notebooks import plot_groups_metrics
from synthesizer import Synthesizer

In [3]:
#synthesizers_path = glob("../out/synthesizer/*/")
#synthesizers_path.sort()
synthesizers_path = ['../out/synthesizer/mel_synth']

In [4]:
groups_metrics = {}
synthesizers_loss = {
    "path": [],
    "dataset": [],
    "hidden_layers": [],
    "dropout_p": [],
    "learning_rate": [],
    "final_loss": [],
}

for synthesizer_path in synthesizers_path:
    synthesizer = Synthesizer.reload(synthesizer_path, load_nn=False)
    config = synthesizer.config
    with open("%s/metrics.pickle" % synthesizer_path, "rb") as f:
        metrics = pickle.load(f)
    final_loss = min(metrics["validation"]["total"])
    
    synthesizers_loss["path"].append(synthesizer_path[-5:])
    synthesizers_loss["dataset"].append(config['dataset']['name'])
    synthesizers_loss["hidden_layers"].append(f"{len(config['model']['hidden_layers'])}x{config['model']['hidden_layers'][0]}")
    synthesizers_loss["dropout_p"].append(config['model']['dropout_p'])
    synthesizers_loss["learning_rate"].append(config['training']['learning_rate'])
    synthesizers_loss["final_loss"].append(final_loss)
    
    group_name = "\n".join((
        f"{config['dataset']['name']}",
        f"hidden_layers={len(config['model']['hidden_layers'])}x{config['model']['hidden_layers'][0]}",
    ))
    
    if group_name not in groups_metrics:
        groups_metrics[group_name] = {}
    groups_metrics[group_name][synthesizer_path] = metrics

synthesizers_loss = pd.DataFrame(synthesizers_loss)

In [7]:
synthesizers_loss.sort_values("final_loss").head(20)

,path,dataset,hidden_layers,dropout_p,learning_rate,final_loss
0,synth,pb2009,4x512,0.05,0.0005,0.432402


In [6]:
metrics_name = [
    "total",
]

def show_metrics(split_name="validation"):
    plot_groups_metrics(groups_metrics, metrics_name, split_name)
display(ipw.interactive(show_metrics, split_name=["train", "validation"]))

interactive(children=(Dropdown(description='split_name', index=1, options=('train', 'validation'), value='vali…